In [8]:

from src.iduedu.modules.intermodal_builder import get_intermodal_graph


intermodal = get_intermodal_graph(osm_id=1114252)

2024-09-01 14:13:35.023 | DEBUG    | iduedu.modules.downloaders:get_boundary_by_osm_id:20 - Downloading territory bounds with osm_id <1114252> ...
2024-09-01 14:13:35.154 | INFO     | src.iduedu.modules.intermodal_builder:get_intermodal_graph:31 - Started downloading and parsing walk graph...
2024-09-01 14:13:35.159 | INFO     | src.iduedu.modules.intermodal_builder:get_intermodal_graph:35 - Started downloading and parsing public trasport graph...
2024-09-01 14:13:40.944 | INFO     | src.iduedu.modules.intermodal_builder:get_intermodal_graph:37 - Public trasport graph done!
2024-09-01 14:13:46.202 | INFO     | src.iduedu.modules.intermodal_builder:get_intermodal_graph:39 - Walk graph done!
2024-09-01 14:13:46.482 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:12 - Composing intermodal graph...
2024-09-01 14:13:47.758 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:140 - Done composing!


In [6]:
from src.iduedu.modules.public_transport_builder import get_all_public_transport_graph

transport = get_all_public_transport_graph(osm_id=3955288)

2024-09-01 13:58:14.348 | DEBUG    | src.iduedu.modules.downloaders:get_boundary_by_osm_id:20 - Downloading territory bounds with osm_id <3955288> ...


2024-09-01 13:58:17.623 | INFO     | src.iduedu.utils.utils:estimate_crs_for_bounds:50 - Estimated CRS for territory EPSG:32638


Parsing routes:   0%|          | 0/157 [00:00<?, ?it/s]

In [4]:
intermodal.graph

{'crs': 32638, 'walk_speed': 83.33333333333333}

In [9]:
import networkx as nx
def convert(val):
    return str(val)

nx.write_gml(intermodal, 'intermodal_vaska.gml', convert)

In [98]:
import numpy as np
%load_ext autoreload
%autoreload 2
from src.iduedu.modules.downloaders import get_routes_by_terr_name
from src.iduedu.modules.public_transport_builder import get_single_public_transport_graph, \
    get_all_public_transport_graph
from src.iduedu.modules.drive_walk_builder import get_drive_graph, get_walk_graph
import pandas as pd
import geopandas as gpd

# g = get_single_public_transport_graph('subway', osm_id=3767134)
transport = get_all_public_transport_graph(osm_id=1114252, clip_by_bounds=False)
# drive = get_drive_graph(osm_id=2725551)

# g = get_single_public_transport_graph('bus', osm_id=3767134)

2024-08-30 14:51:23.798 | DEBUG    | src.iduedu.modules.downloaders:get_boundary_by_osm_id:20 - Downloading territory bounds with osm_id <1114252> ...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2024-08-30 14:51:27.155 | INFO     | src.iduedu.utils.utils:estimate_crs_for_bounds:50 - Estimated CRS for territory EPSG:32636


Parsing routes data...:   0%|          | 0/57 [00:00<?, ?it/s]

In [99]:
walk = get_walk_graph(osm_id=1114252)

2024-08-30 14:51:32.247 | DEBUG    | iduedu.modules.downloaders:get_boundary_by_osm_id:20 - Downloading territory bounds with osm_id <1114252> ...
2024-08-30 14:51:32.375 | DEBUG    | src.iduedu.modules.drive_walk_builder:get_walk_graph:179 - Downloading walk graph from OSM ...
2024-08-30 14:51:42.583 | INFO     | iduedu.utils.utils:estimate_crs_for_bounds:50 - Estimated CRS for territory EPSG:32636
2024-08-30 14:51:42.584 | DEBUG    | src.iduedu.modules.drive_walk_builder:get_walk_graph:183 - Calculating the weights of the graph ...


In [100]:
import networkx as nx

num_nodes_G1 = len(transport.nodes)

# Создаём отображение (relabel) для нод в первом графе (нумерация с 0)
mapping_G1 = {node: idx for idx, node in enumerate(transport.nodes)}

# Создаём отображение для нод во втором графе (нумерация начиная с num_nodes_G1)
mapping_G2 = {node: idx + num_nodes_G1 for idx, node in enumerate(walk.nodes)}

# Применяем relabel для графов
transport: nx.MultiDiGraph = nx.relabel_nodes(transport, mapping_G1)
walk: nx.MultiDiGraph = nx.relabel_nodes(walk, mapping_G2)

In [101]:
import geopandas as gpd
import networkx as nx
from shapely import Point

platforms = pd.DataFrame.from_dict(dict(transport.nodes(data=True)), orient='index')
platforms = platforms[platforms['desc'] == 'platform']
platforms['geometry'] = platforms.apply(lambda x: Point(x.x, x.y), axis=1)
platforms = gpd.GeoDataFrame(platforms, crs=walk.graph['crs'])
walk_edges = gpd.GeoDataFrame(nx.to_pandas_edgelist(walk, source='u', target='v', edge_key='k'), crs=walk.graph['crs'])

In [102]:
projection_join = platforms.sjoin_nearest(walk_edges, max_distance=20, distance_col='dist')

In [103]:
points_grouped_by_edge = projection_join.reset_index().groupby('index_right', group_keys=True).agg(
    {'index': tuple, "geometry": tuple, "u": 'first', "v": 'first', "k": 'first'})

In [104]:
g1 = walk.copy()
g2 = transport.copy()


In [144]:
walk = g1.copy()
transport = g2.copy()

In [145]:
from shapely.ops import substring
from shapely import LineString

try:
    speed = walk.graph['walk_speed']
except KeyError:
    print("Нету скорости")  # TODO посчитать примерную скорость по length timemin для любой эджи
    speed = 83

edges_to_del = []

for _, row in points_grouped_by_edge.iterrows():
    u, v, k = row[['u', 'v', 'k']]
    edge: LineString = walk_edges.loc[row.name].geometry

    if len(row['index']) == 1:
        dist = edge.project(row.geometry[0])
        projected_point = edge.interpolate(dist)
        if dist == 0:
            #Если платформа проецируется на начало эджа
            mapping = {u: row['index'][0]}
            nx.relabel_nodes(walk, mapping, copy=False)
            
        elif dist == edge.length:
            # Если на конец
            mapping = {v: row['index'][0]}
            nx.relabel_nodes(walk, mapping, copy=False)

        else:

            line1 = substring(edge, 0, dist)
            line2 = substring(edge, dist, edge.length)
            # Убираем старую эджу и добавляем новые, в серединке нода - наша платформа для соединения
            # walk.remove_edge(u, v, k)
            edges_to_del.append((u, v, k))
            walk.add_node(row['index'][0], x=projected_point.x, y=projected_point.y)
            walk.add_edge(u, row['index'][0], geometry=line1, length=round(line1.length, 3),
                          time_min=round(line1.length / speed, 3),
                          type='walk')  # TODO заменить type на walk
            walk.add_edge(row['index'][0], v, geometry=line2, length=round(line2.length, 3),
                          time_min=round(line2.length / speed, 3), type='walk')

    else:
        # walk.remove_edge(u, v, k)

        edges_to_del.append((u, v, k))
        dist_project = [(edge.project(platform), edge.interpolate(edge.project(platform)), ind) for ind, platform in
                        zip(row['index'], row.geometry)]
        dist_project.sort(key=lambda x: x[0])
        last_dist = 0
        last_u = u
        for dist, projected_point, cur_index in dist_project:

            if dist == 0:
                #Если платформа проецируется на начало эджа
                mapping = {u: cur_index}
                nx.relabel_nodes(walk, mapping, copy=False)


            elif dist == edge.length:
                # Если на конец
                mapping = {v: cur_index}
                nx.relabel_nodes(walk, mapping, copy=False)


            else:
                # По очереди добавляем ноды/линии

                line = substring(edge, last_dist, dist)
                #
                walk.add_node(cur_index, x=projected_point.x, y=projected_point.y)
                walk.add_edge(last_u, cur_index, geometry=line, length=round(line.length, 3),
                              time_min=round(line.length / speed, 3),
                              type='walk')  # TODO заменить type на walk

                last_u = cur_index
            last_dist = dist
        # Если последняя остановка спроецировалась не на конец эджа, надо добавить остаток 
        if last_dist != edge.length:
            line = substring(edge, last_dist, edge.length)
            walk.add_edge(last_u, v, geometry=line, length=round(line.length, 3),
                          time_min=round(line.length / speed, 3),
                          type='walk')

In [146]:
walk.remove_edges_from(edges_to_del)

In [147]:
def convert(val):
    return str(val)


compose = nx.compose(nx.MultiDiGraph(transport), walk)

nx.write_gml(compose, 'compose.gml', convert)

In [97]:
import osmnx as ox
import networkx as nx
import folium

nodes, edges = ox.graph_to_gdfs(compose)
m1 = edges.explore(column='type', tiles='CartoDB positron')
nodes.explore(column='desc', cmap='plasma', m=m1)
folium.LayerControl().add_to(m1)
m1.save('kek.html')

In [ ]:
import networkx as nx
from shapely import Polygon
from src.iduedu.modules.downloaders import get_boundary_by_osm_id
from src.iduedu.utils.utils import estimate_crs_for_bounds


def clip_nx_graph(graph: nx.Graph, polygon: Polygon):
    local_crs = graph.graph['crs']


def reproject_nx_graph(graph: nx.Graph, crs=None):
    """

    :param graph:
    :param crs: by default None, will be determine automatically
    :return:
    """
    print(1)


poly: Polygon = get_boundary_by_osm_id(3767134)
estimate_crs_for_bounds(*poly.bounds)

In [ ]:
import geopandas as gpd

gpd.GeoDataFrame(geometry=[poly], crs=4326).estimate_utm_crs()

In [ ]:
import networkx as nx

nx.write_gml(g2, 'test_saratov_all.gml')

In [ ]:
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

connect_points = [point for coords in connected_ways for point in (coords[0], coords[-1])]
distances = cdist(connect_points, connect_points)
n = distances.shape[0]
mask = (np.arange(n)[:, None] // 2) == (np.arange(n) // 2)
distances[mask] = np.inf

indexes = []


def relative_point(point: int):
    rel_point = point // 2 * 2
    return rel_point if rel_point != point else rel_point + 1


lines = []

connected_line = []
debug = []
first_con_1, first_con_2 = np.unravel_index(np.argmin(distances), distances.shape)
distances[first_con_1, :] = np.inf
distances[first_con_2, :] = np.inf
distances[:, first_con_2] = np.inf
distances[:, first_con_1] = np.inf
first_con_1_rel, first_con_2_rel = [relative_point(x) for x in (first_con_1, first_con_2)]
distances[first_con_1_rel, first_con_2_rel] = np.inf
distances[first_con_2_rel, first_con_1_rel] = np.inf
indexes.append((first_con_1, first_con_2))

line1, line2 = first_con_1 // 2, first_con_2 // 2

lines.append(line1)
lines.append(line2)
if first_con_1 % 2 == 1:
    connected_line += connected_ways[line1]
    debug.append(connected_ways[line1])
else:
    connected_line += connected_ways[line1][::-1]
    debug.append(connected_ways[line1][::-1])

if first_con_2 % 2 == 0:
    connected_line += connected_ways[line2]
    debug.append(connected_ways[line2])
else:
    connected_line += connected_ways[line2][::-1]
    debug.append(connected_ways[line2][::-1])

extreme_points = [first_con_1_rel, first_con_2_rel]

print(f'extrem_from_start {extreme_points}')

for i in range(len(connected_ways) - 2):
    position, ind = np.unravel_index(np.argmin(distances[extreme_points]), (2, n))
    next_con = (extreme_points[position], ind)
    rel_point = relative_point(next_con[1])
    distances[:, rel_point] = np.inf
    line = ind // 2
    lines.append(line)
    if position == 0:
        connected_line = connected_ways[line] + connected_line if ind % 2 == 1 else (
                connected_ways[line][::-1] + connected_line)
        debug.insert(0, connected_ways[line]) if ind % 2 == 1 else debug.insert(0, connected_ways[line][::-1])
        indexes.insert(0, next_con[::-1])
        extreme_points = [rel_point, extreme_points[1]]
    else:
        connected_line = connected_line + connected_ways[line] if ind % 2 == 0 else (
                connected_line + connected_ways[line][::-1])
        debug.append(connected_ways[line]) if ind % 2 == 1 else debug.append(connected_ways[line][::-1])
        indexes.append(next_con)
        extreme_points = [extreme_points[0], rel_point]

    distances[next_con[0], :] = np.inf
    distances[next_con[1], :] = np.inf
    distances[:, next_con[0]] = np.inf
    distances[:, next_con[1]] = np.inf
    rel_point_2 = relative_point(next_con[0])
    distances[rel_point, rel_point_2] = np.inf
    distances[rel_point_2, rel_point] = np.inf
    print('---------------')
    print(f'index: {i}')
    print(indexes)
    print(f'extreme_points {extreme_points}')
    print(f'added con {next_con}')


In [ ]:
connected_ways

In [ ]:
from shapely import LineString
import geopandas as gpd

gpd.GeoDataFrame(geometry=[LineString(new_connected_ways)], crs=32635).explore(tiles='CartoDB positron')

In [ ]:
t = [1, 2]
[(x // 2) * 2 if ((x // 2) * 2 != x) else ((x // 2) * 2 + 1) for x in t]

In [ ]:
%load_ext autoreload
%autoreload 2
from iduedu.modules.graph_builder import get_all_public_transport_graph

res = get_all_public_transport_graph(osm_id=3767134)

In [ ]:
import networkx as nx

nx.write_gml(res, 'spb+lo_iduedu.gml')

In [ ]:
from shapely import from_wkt
import osmnx as ox
import geopandas as gpd
import networkx as nx


def nx_to_gdf(
        graph: nx.MultiDiGraph, nodes: bool = False, edges: bool = False, node_geometry=False, fill_edge_geometry=False
) -> gpd.GeoDataFrame | tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    for _, _, data in graph.edges(data=True):
        if "geometry" in data:
            data["geometry"] = from_wkt(str(data["geometry"]))

    if edges and nodes:
        gdf_graph_nodes, gdf_graph_edges = ox.graph_to_gdfs(
            graph, nodes=True, edges=True, node_geometry=node_geometry, fill_edge_geometry=fill_edge_geometry
        )
        gdf_graph_edges = gdf_graph_edges.reset_index()
        return gdf_graph_nodes, gdf_graph_edges
    elif edges:
        gdf_graph_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True, fill_edge_geometry=fill_edge_geometry)
        gdf_graph_edges = gdf_graph_edges.reset_index()
        return gdf_graph_edges
    elif nodes:
        gdf_graph_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False, node_geometry=node_geometry)
        return gdf_graph_nodes
    raise ValueError("You must specify either nodes or edges as True.")


g.graph['crs'] = 32638
nodes, edges = nx_to_gdf(nx.MultiDiGraph(g), nodes=True, edges=True, node_geometry=True)

In [ ]:
m1 = edges.explore(column='route', tiles='CartoDB positron')
# nodes.explore(m=m1)
m1

In [ ]:
t = data_geom.iloc[3]
platforms = gpd.GeoDataFrame(geometry=t['platforms'], crs=local_crs)
stops = gpd.GeoDataFrame(geometry=t['stops'], crs=local_crs)

connection = gpd.sjoin_nearest(platforms, stops)
connection

In [ ]:
from shapely.ops import substring

line = LineString([(0, 0), (2, 2), (5, 5), (10, 0)])

start_distance = 4
end_distance = 6

sub_line = substring(line, 4, 10)
line

In [ ]:
m1 = gpd.GeoDataFrame(geometry=[LineString(x) for x in data_geom['path'].iloc[2:3]], crs=local_crs).explore()
gpd.GeoDataFrame(geometry=graph_gdf.iloc[2], crs=local_crs).explore(m=m1, color='red')

In [ ]:
from shapely import LineString
import geopandas as gpd

m1 = gpd.GeoDataFrame(geometry=[LineString(x) for x in data_geom['path']], crs=local_crs).reset_index().explore(
    column='index', tiles='CartoDB positron')
# gpd.GeoDataFrame(data_geom.iloc[1]['platforms'], crs=4326).reset_index().explore(m=m1, color='red')
m1

In [ ]:
t = [(1, 6), (0, 3), (2, 5)]
for i in range(3):


In [ ]:
test = pd.DataFrame(data_osm.iloc[2]['members'])
test = test[(test['type'] == 'node') & (test['role'].isin(['platform_entry_only', 'platform', 'platform_exit_only']))]
test['geometry'] = test.apply(lambda x: (Point(x["lon"], x["lat"])), axis=1)
test.reset_index(drop=True, inplace=True)
gdf = gpd.GeoDataFrame(test.reset_index(), crs=4326)
gdf.geometry = gdf.geometry.buffer(0.0001)
gdf.explore()

In [ ]:
test = pd.DataFrame(data_osm.iloc[1]['members'])
test = test[test['type'] == 'way']
test = test["geometry"]
test = test.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x]))
# gdf = gpd.GeoDataFrame(test.reset_index(),geometry='geometry',crs=4326)
# gdf.geometry = gdf.geometry.buffer(0.0001)
# gdf.explore()
test

In [ ]:
test

#### ПОПЫТКА СОЕДИНЯТЬ ДОРОГИ 

In [ ]:
from scipy.spatial import cKDTree
from shapely import LineString, Point
import geopandas as gpd

PLATFORM_ROLES = []


def parse_overpass_route_response(loc: dict):
    if 'ref' in loc['tags'].keys():
        transport_name = loc['tags']['ref']
    else:
        transport_name = loc['tags']['name']
    route = pd.DataFrame(loc["members"])

    ways = route[route["type"] == "way"]

    if len(ways) > 0:
        ways = ways["geometry"].reset_index(drop=True)
        ways = ways.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x])).tolist()
        if ways[0][0] == ways[1][0] or ways[0][0] == ways[1][-1]:
            complete_line = ways[0][::-1]
        else:
            complete_line = ways[0]
        for coords in ways[1:]:
            if complete_line[-1] == coords[0]:
                complete_line += coords[1:]
            elif complete_line[-1] == coords[-1]:
                complete_line += coords[::-1][1:]
            else:
                if coords[0] == coords[-1]:
                    continue
                else:
                    tree = cKDTree([coords[0], coords[-1]])
                    _, ind = tree.query(complete_line[-1])
                    if ind == 1:
                        coords = coords[::-1]
                    complete_line += coords
        # print('^^^^^^^^^^')

    else:
        complete_line = None
    if "node" in route["type"].unique():
        platforms = route[(route["type"] == "node")].apply(lambda x: Point(x["lon"], x["lat"]), axis=1)
    else:
        platforms = None

    return pd.Series({"way": complete_line, "platforms": platforms, 'name': transport_name})


data_geom = data_osm.apply(parse_overpass_route_response, axis=1, result_type='expand')
data_geom

In [ ]:
test = [LineString(
    [(46.0046822, 51.5426408), (46.0046214, 51.5425402), (46.0042506, 51.541918), (46.0038134, 51.5411845),
     (46.0036437, 51.5408997), (46.0035859, 51.5408011), (46.0035352, 51.5407158), (46.0031053, 51.5399929),
     (46.0028598, 51.5395801), (46.0026303, 51.5391941), (46.0025706, 51.5390938), (46.0023997, 51.5391331),
     (46.00211, 51.5391985), (46.0019414, 51.5392372), (46.0008809, 51.5394906), (46.00081, 51.5395332),
     (46.000665, 51.5396234), (46.0006024, 51.5396705), (46.0005255, 51.5397869), (46.000459, 51.5399285),
     (46.0005332, 51.54006), (46.0007185, 51.5403886), (46.0008003, 51.5405164), (46.0008441, 51.5405848)]), LineString(
    [(46.0046822, 51.5426408), (46.0048226, 51.5426046), (46.0048853, 51.5425884), (46.0065015, 51.5422214),
     (46.0067008, 51.5421762), (46.007038, 51.5420996), (46.0071807, 51.5420672)]), Point(46.0046822, 51.5426408)]

gpd.GeoDataFrame(geometry=test, crs=4326).explore()

In [ ]:
gpd.GeoDataFrame(geometry=data_geom['way'].apply(LineString), crs=4326).reset_index().explore(column='index')

In [ ]:
print(data_geom['way'].apply(LineString).iloc[1])

In [ ]:
data_geom.iloc[16]['way']

In [ ]:
ways = data_geom.iloc[16]['way']
ways = ways["geometry"].reset_index(drop=True)
lines = ways.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x])).tolist()
if lines[0][0] == lines[1][0]:
    complete_line = lines[0][::-1]
else:
    complete_line = lines[0]
for coords in lines[1:]:
    if complete_line[-1] == coords[0]:
        complete_line += coords[1:]
    elif complete_line[-1] == coords[-1]:
        complete_line += coords[::-1][1:]
    else:
        print('\n\nWTF\n\n')



In [ ]:
gpd.GeoDataFrame(geometry=[LineString(complete_line)], crs=4326).explore()

In [ ]:

from shapely import MultiPoint

gpd.GeoDataFrame(data=[{'kek': i} for i in data_geom.index.tolist()],
                 geometry=data_geom['platforms'].apply(lambda x: MultiPoint(x)), crs=4326).explore(column='kek')

In [ ]:

data_geom.index.tolist()

## Download from microsoft ML footprint

"""
This snippet demonstrates how to access and convert the buildings
data from .csv.gz to geojson for use in common GIS tools. You will
need to install pandas, geopandas, and shapely.
"""

import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

location = 'Russia'

dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
greece_links = dataset_links[dataset_links.Location == location]


def convert_to_bytes(size):
    size = str(size)
    if 'KB' in size:
        return float(size.replace('KB', '')) * 1024
    elif 'MB' in size:
        return float(size.replace('MB', '')) * 1024 * 1024
    elif 'B' in size:
        return float(size.replace('B', ''))
    else:
        return float(size)


greece_links['Size_in_bytes'] = greece_links['Size'].apply(convert_to_bytes)
greece_links = greece_links.sort_values(by='Size_in_bytes', ascending=False)
greece_links = greece_links.head(10)

greece_links = greece_links
for _, row in greece_links.iterrows():
    df = pd.read_json(row.Url, lines=True)
    df['geometry'] = df['geometry'].apply(shape)
    gdf = gpd.GeoDataFrame(df, crs=4326)
    gdf.to_file(f"{row.QuadKey}.geojson", driver="GeoJSON")



location = 'Russia'

dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
greece_links = dataset_links[dataset_links.Location == location]


def convert_to_bytes(size):
    size = str(size)
    if 'KB' in size:
        return float(size.replace('KB', '')) * 1024
    elif 'MB' in size:
        return float(size.replace('MB', '')) * 1024 * 1024
    elif 'B' in size:
        return float(size.replace('B', ''))
    else:
        return float(size)


greece_links['Size_in_bytes'] = greece_links['Size'].apply(convert_to_bytes)
greece_links = greece_links.sort_values(by='Size_in_bytes', ascending=False)

## Для вшэ временно тут
import math

import geopandas as gpd
f1 = 'Зоны видимости школ (без деревьев).geojson'
f2 = 'Зоны видимости школ (с деревьями).geojson'

d1 = gpd.read_file(f1).to_crs(32636)
d2 = gpd.read_file(f2).to_crs(32636)

d1.drop(columns=['area','perimeter'],inplace=True)
d2.drop(columns=['area','perimeter'],inplace=True)

import pandas as pd
schools = gpd.read_file('Полигоны школ.geojson').to_crs(32636)
schools['ID школы'] = schools['ID школы'].astype(int)
schools['Площадь школы(м2)'] = round(schools.geometry.area,2)
schools['school_geom'] = schools.geometry
d1_ =  pd.merge(d1,schools[['Площадь школы(м2)','ID школы','school_geom']],on='ID школы')
d2_ =  pd.merge(d2,schools[['Площадь школы(м2)','ID школы','school_geom']],on='ID школы')

import math

d1_['Площадь видимости(м2)'] = round(d1_.geometry.area,2)
d2_['Площадь видимости(м2)'] = round(d2_.geometry.area,2)

d1_['Коэффициент видимости'] = d1_.apply(lambda x: round(x.geometry.area / x['school_geom'].buffer(1000).area,5),axis=1)
d2_['Коэффициент видимости'] = d2_.apply(lambda x: round(x.geometry.area / x['school_geom'].buffer(1000).area,5),axis=1)

d1_.drop(columns='school_geom').to_file(f1)
d2_.drop(columns='school_geom').to_file(f2)